In [ ]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"

import os
import sys

import random
import warnings
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import ShuffleSplit
import cv2
from scipy import ndimage
import time
import datetime

# get package versions
def get_version(*vars):
    for var in vars:
        module = __import__(var)    
        print ('%s: %s' %(var,module.__version__))
    
# package version    
get_version('keras','numpy','matplotlib','cv2','theano')


## Settings

In [ ]:
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = '../data/stage1_train/'
TEST_PATH = '../data/stage1_test/'

# normalization type
norm_type='zeroMeanUnitStd'

netinfo='trainTest2'
initialLearningRate=3e-4


# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

## Utilities

In [ ]:
# calcualte dice
def calc_dice(X,Y,d=0):
    N=X.shape[d]    
    # intialize dice vector
    dice=np.zeros([N,1])

    for k in range(N):
        x=X[k,0] >.5 # convert to logical
        y =Y[k,0]>.5 # convert to logical

        # number of ones for intersection and union
        intersectXY=np.sum((x&y==1))
        unionXY=np.sum(x)+np.sum(y)

        if unionXY!=0:
            dice[k]=2* intersectXY/(unionXY*1.0)
            #print 'dice is: %0.2f' %dice[k]
        else:
            dice[k]=1
            #print 'dice is: %0.2f' % dice[k]
        #print 'processing %d, dice= %0.2f' %(k,dice[k])
    return np.mean(dice),dice

def preprocess(X,xnormType=None):
    if xnormType=='minus1plus1':
        X=X.astype('float32')
        X/=np.max(X)
        X-=0.5
        X=X*2
    elif xnormType=='zeroMeanUnitStd':
        X=X.astype('float32')
        # we do this per channel
        for c in range(X.shape[1]):
            X[:,c]-=np.mean(X[:,c])
            stdXc=np.std(X[:,c])
            if stdXc>0.0:
                X[:,c]/=stdXc
    else:
        print('no normalization!')
    return X

def array_stats(X):
    X=np.asarray(X)
    print ('array shape: ',X.shape, X.dtype)
    #print 'min: %.3f, max:%.3f, avg: %.3f, std:%.3f' %(np.min(X),np.max(X),np.mean(X),np.std(X))
    print ('min: {}, max: {}, avg: {:.3}, std:{:.3}'.format( np.min(X),np.max(X),np.mean(X),np.std(X)))

def resizeY(Y,origHW):
    # Y shape is N*1*H*W
    N=Y.shape[0] 
    Yr=[]
    for k in range(N):
        temp=resize(Y[k,0], (origHW[k,0], origHW[k,1]), mode='constant', preserve_range=True)
        Yr.append(temp)
    return Yr
    
def loadData0(ids,path2data):
    print('loading '+path2data)
    # Get and resize train images and masks
    X = np.zeros((len(ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y = np.zeros((len(ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    origHW=np.zeros((len(ids),3),dtype='uint16') # store original dimension for later use
    
    print('Getting and resizing images and masks ... ')
    #sys.stdout.flush()
    for n, id_ in enumerate(ids):
        path = path2data + id_   
        #print(path)
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
        h,w,c=img.shape
        origHW[n]=img.shape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X[n] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        try:
            for mask_file in next(os.walk(path + '/masks/'))[2]:
                mask_ = imread(path + '/masks/' + mask_file)
                mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                              preserve_range=True), axis=-1)
                mask = np.maximum(mask, mask_)
            Y[n] = mask
        except:
            Y=[]
    if len(Y):
        Y=np.transpose(Y,(0,3,1,2))
    return np.transpose(X,(0,3,1,2)),Y,origHW

def loadData(path2h5,data_type='train'):
    if not os.path.exists(path2h5):
        X_train,Y_train,HW_train=loadData0(train_ids,TRAIN_PATH)
        h5file=h5py.File(path2h5,'w-')
        h5file['X_train']=X_train
        h5file['Y_train']=Y_train
        h5file['HW_train']=HW_train
            
        X_test,Y_test,HW_test=loadData0(test_ids,TEST_PATH)
        h5file['X_test']=X_test
        h5file['Y_test']=Y_test
        h5file['HW_test']=HW_test
    else:
        print('loading '+ path2h5)
        h5file=h5py.File(path2h5,'r')
        X_train=h5file['X_train']
        Y_train=h5file['Y_train']
        HW_train=h5file['HW_train']
        X_test=h5file['X_test']
        Y_test=h5file['Y_test']
        HW_test=h5file['HW_test']
    if data_type=="train":
        return X_train,Y_train,np.array(HW_train,'uint16')
    elif data_type=="leader":
        return X_test,Y_test,np.array(HW_train,'uint16')

    
    
# train test model
def train_test_model(X_train,y_train,X_test,y_test,params_train):
    foldnm=params_train['foldnm']  
    pre_train=params_train['pre_train'] 
    batch_size=params_train['batch_size'] 
    augmentation=params_train['augmentation'] 
    path2weights=params_train['path2weights'] 
    path2model=params_train['path2model'] 
    norm_type=params_train['norm_type'] 
    
    print('batch_size: %s, Augmentation: %s' %(batch_size,augmentation))
    
    print 'fold %s training in progress ...' %foldnm
    # load last weights
    if pre_train== True:
        if  os.path.exists(path2weights):
            model.load_weights(path2weights)
            print 'previous weights loaded!'
        else:
            raise IOError('weights does not exist!!!')
    else:
        if  os.path.exists(path2weights):
            model.load_weights(path2weights)
            print (path2weights)
            print ('previous weights loaded!')
            train_status='previous weights'
            return train_status
    
    # path to csv file to save scores
    path2scorescsv = weightfolder+'/scores.csv'
    first_row = 'train,test'
    with open(path2scorescsv, 'w+') as f:
        f.write(first_row + '\n')
           
    # Fit the model
    start_time=time.time()
    scores_test=[]
    scores_train=[]
    if params_train['loss']=='dice': 
        best_score = 0
        previous_score = 0
    else:
        best_score = 1e6
        previous_score = 1e6
    patience = 0
    
    # convert class vectors to binary class matrices
    train_generator = generator(X_train,y_train,batch_size)
    
    for epoch in range(params_train['nbepoch']):
    
        print ('epoch: %s,  Current Learning Rate: %.1e' %(epoch,model.optimizer.lr.get_value()))
        #seed = np.random.randint(0, 999999)
    
        if augmentation:
            model.fit_generator(train_generator, steps_per_epoch=len(xtr)/batch_size, epochs=1,verbose=0)            
        else:
            hist=model.fit(preprocess(X_train,norm_type), y_train, batch_size=batch_size,epochs=1, verbose=0)
            
        # evaluate on test and train data
        score_test=model.evaluate(preprocess(X_test,norm_type),y_test,verbose=0)
        score_train=np.mean(hist.history['loss'])
       
        print ('score_train: %s, score_test: %s' %(score_train,score_test))
        scores_test=np.append(scores_test,score_test)
        scores_train=np.append(scores_train,score_train)    

        # check for improvement    
        if (score_test<=best_score):
            print ("!!!!!!!!!!!!!!!!!!!!!!!!!!! viva, improvement!!!!!!!!!!!!!!!!!!!!!!!!!!!") 
            best_score = score_test
            patience = 0
            model.save_weights(path2weights)  
            model.save(path2model)
            
        # learning rate schedule
        if score_test>previous_score:
            #print "Incrementing Patience."
            patience += 1

        # learning rate schedule                
        if patience == params_train['max_patience']:
            params_train['learning_rate'] = params_train['learning_rate']/2
            print ("Upating Current Learning Rate to: ", params_train['learning_rate'])
            model.optimizer.lr.set_value(params_train['learning_rate'])
            print ("Loading the best weights again. best_score: ",best_score)
            model.load_weights(path2weights)
            patience = 0
        
        # save current test score
        previous_score = score_test    
        
        # store scores into csv file
        with open(path2scorescsv, 'a') as f:
            string = str([score_train,score_test])
            f.write(string + '\n')
           
    
    print ('model was trained!')
    elapsed_time=(time.time()-start_time)/60
    print ('elapsed time: %d  mins' %elapsed_time)      

    # train test progress plots
    plt.figure(figsize=(10,10))
    plt.plot(scores_test)
    plt.plot(scores_train)
    plt.title('train-validation progress',fontsize=20)
    plt.legend(('test','train'),fontsize=20)
    plt.xlabel('epochs',fontsize=20)
    plt.ylabel('loss',fontsize=20)
    plt.grid(True)
    plt.savefig(weightfolder+'/train_val_progress.png')
    plt.show()
    
    print 'training completed!'
    train_status='completed!'
    return train_status    

def grays_to_RGB(img):
    # turn 2D grayscale image into grayscale RGB
    return np.dstack((img, img, img))


def image_with_mask(img, mask,color=(0,255,0)):
    mask=np.asarray(mask,dtype='uint8') 
    
    if len(img.shape)==2:
        img_color = grays_to_RGB(img)
    else:
        img_color =img

    mask2=mask[:,:,0]
    for c1 in range(mask.shape[2]):
        mask2=np.logical_or(mask2,mask[:,:,c1])
    mask2=np.array(255*mask2,'uint8')
        
    mask_edges = cv2.Canny(mask2, 100, 200) > 0
    #plt.imshow(mask_edges)
    maximg=np.max(img)
    img_color[mask_edges, 0] = maximg*color[0]  # set channel 0 to bright red, green & blue channels to 0
    img_color[mask_edges, 1] = maximg*color[1]
    img_color[mask_edges, 2] = maximg*color[2]
    return img_color

def disp_img_2masks(img,mask1=None,mask2=None,r=1,c=1,d=0,indices=None):
    if mask1 is None:
        mask1=np.zeros(img.shape,dtype='uint8')
    else:
        mask1=np.array(mask1,dtype='uint8')
    if mask2 is None:
        mask2=np.zeros(img.shape,dtype='uint8')
    else:        
        mask2=np.array(mask2,dtype='uint8')    
        
    N=r*c    
    if d==2:
        # convert to N*C*H*W
        img=np.transpose(img,(2,0,1))
        img=np.expand_dims(img,axis=1)
        
        mask1=np.transpose(mask1,(2,0,1))
        mask1=np.expand_dims(mask1,axis=1)

        mask2=np.transpose(mask2,(2,0,1))
        mask2=np.expand_dims(mask2,axis=1)
        
    if indices is None:    
        indices=np.random.randint(img.shape[0],size=N)
    
    # collect images and masks
    I1=[np.transpose(img[i],(1,2,0)) for i in indices]
    M1=[np.transpose(mask1[i],(1,2,0)) for i in indices]
    M2=[np.transpose(mask2[i],(1,2,0)) for i in indices]
    
    C1=(0,255,0)
    C2=(255,0,0)
    for k in range(N):    
        imgmask=image_with_mask(I1[k],M1[k],C1)
        imgmask=image_with_mask(imgmask,M2[k],C2)
        plt.subplot(r,c,k+1)
        plt.imshow(imgmask)
        plt.title(indices[k])
    plt.show()            
    


## loading data

In [ ]:
path2h5='../data/trainTestH'+str(IMG_HEIGHT)+'W'+str(IMG_WIDTH)+'.hdf5'
X,Y,HW_train=loadData(path2h5,'train')
array_stats(X)
array_stats(Y)
array_stats(HW_train)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.hist(HW_train[:,0],100)
plt.xlim([0,400])
plt.xticks(range(0,400,32))
plt.title('H')
plt.subplot(1,2,2)
plt.hist(HW_train[:,1],100)
plt.xlim([0,400])
plt.xticks(range(0,400,32))
plt.title('W')
plt.show()

## Display data

In [ ]:
plt.figure(figsize=(15,10))
disp_img_2masks(img=X,mask1=Y,r=2,c=3);

In [ ]:
from image import ImageDataGenerator

# random data generator
datagen = ImageDataGenerator(featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.01,
        zoom_range=0.01,
        channel_shift_range=0.0,
        fill_mode='constant',
        cval=0.0,
        horizontal_flip=True,
        vertical_flip=True,)
        #data_format="channels_first") 

In [ ]:
def iterate_minibatches(inputs1 , targets,  batchsize, shuffle=True, augment=True):
    assert len(inputs1) == len(targets)
    if augment==True:
        if shuffle:
            indices = np.arange(len(inputs1))
            np.random.shuffle(indices)
        for start_idx in range(0, len(inputs1) - batchsize + 1, batchsize):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batchsize]
            else:
                excerpt = slice(start_idx, start_idx + batchsize)
            x = inputs1[excerpt]
            y = targets[excerpt] 
            for  xxt,yyt in datagen.flow(x, y , batch_size=x.shape[0]):
                x = xxt.astype(np.float32) 
                y = yyt 
                break
    else:
        x=inputs1
        y=targets

    #yield np.array(x,np.uint8), np.array(y, dtype=np.uint8)         
    return np.array(x,np.uint8), np.array(y, dtype=np.uint8) 

In [ ]:
N=X.shape[0]
i=0
array_stats(X)
array_stats(Y)
X_batch,Y_batch=iterate_minibatches(X,Y,N,shuffle=False,augment=True)  
Y_batch=Y_batch[:,0][:,np.newaxis]
array_stats(X_batch)
array_stats(Y_batch)

In [ ]:
plt.figure(figsize=(15,10))
disp_img_2masks(img=X_batch,mask1=Y_batch[:,0][:,np.newaxis],r=2,c=3);

In [ ]:
def gammaAug(images,gamma):
    # images shape: N*C*H*W
    #print ('before gamma:', np.mean(image))
    n1,c1,_,_=images.shape
    for k1 in range(n1):
        #for k2 in range(c1):
        g = (2 * np.random.rand() - 1) * gamma + 1.
        images[k1]=(images[k1]) ** g
    #print ('after gamma:', np.mean(image))
    return images

array_stats(X)
Xgamma=gammaAug(X.value,0.3)
array_stats(Xgamma)

In [ ]:
plt.figure(figsize=(15,10))
disp_img_2masks(img=Xgamma,mask1=Y,r=2,c=3);